In [3]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"

#dataset0 = pd.read_csv(file0)

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'

#features=['draw','horse_winPer',
#                 'jockey_winPer','horse_newDistance','horse_newHorse',
#         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRadio','weightRD']

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'
#'horse_newDistance',,'weightRadio','declaredHorseWeight','addedWeight','horse_newDistance','horse_latestSpeedRate',
#'draw','jockey_winPer','horse_winCount',
features_fea=['horse_winPer','jockey_winPer',
             'horse_newDistance','draw',
              'horse_latestSpeedRate','horseFx', 'horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']

target_fea='lbs'
target=[target_fea]

sql = 'select 1 as const,raceDate,raceSeqOfDay,distance,raceClass, horseId, place,winOdds,finishTime,'
for aFeature in features_fea:
    sql = sql + aFeature +", "
sql = sql+ target_fea+ " from racecard"
#where racedate='20190213'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features_fea,*target]

dataset0 = pd.read_sql(sql,mydb)
#print(dataset0)
allDates =['20190217']
#allDates= dataset0.raceDate.unique()
tableName='racecard'
#dataset0
updatedate='20190219'

 # We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])
features=['const',*features_fea]


NameError: name 'features' is not defined

In [113]:
dataset0.describe()
raceClass=4
distance=1400
dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance)]
#print(dataset_train)
X_train0=dataset_train[features]
X_train0.describe()

y_train=dataset_train[target] #lbw

combine = X_train0.merge(y_train,how='inner',left_index=True,right_index=True)
#print(combine)
combine.corr()
#X_train=preprocessor.fit_transform(X_train0)
#X_train=X_train0
#print(X_train)                         
#import statsmodels.formula.api as sm
#regressor_OLS=sm.OLS(endog=y_train,exog=X_train).fit()
#regressor_OLS.summary()                         

,const,horse_winPer,jockey_winPer,horse_newDistance,draw,horse_latestSpeedRate,lbs
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN
horse_winPer,NaN,1.000000,0.232143,-0.243578,-0.018381,0.287898,-0.210309
jockey_winPer,NaN,0.232143,1.000000,-0.076406,0.013053,0.103245,-0.187592
horse_newDistance,NaN,-0.243578,-0.076406,1.000000,-0.032279,-0.977565,0.053939
draw,NaN,-0.018381,0.013053,-0.032279,1.000000,0.032611,0.058505
horse_latestSpeedRate,NaN,0.287898,0.103245,-0.977565,0.032611,1.000000,-0.092674
lbs,NaN,-0.210309,-0.187592,0.053939,0.058505,-0.092674,1.000000


In [2]:

for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['raceDate']==raceDate)]
    
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()

    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(raceDate,seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw
        
        test_rowsize = X_test0.shape[0] 
        #print(test_rowsize)
        X=pd.concat([X_train0,X_test0])
        #print("=============X========")
        #print(X)
        #print(X.shape)
        #X_arr=preprocessor.fit_transform(X)
        
        #print("=============X_arr========")
        #print(X_arr)
        #print(X_arr.shape)
        
        #X_train=pd.DataFrame(data=X_arr[:-test_rowsize],columns=X_test0.columns)
        #X_test=pd.DataFrame(data=X_arr[-test_rowsize:],columns=X_test0.columns)
        #,X_arr[-test_rowsize:]
        
      #  print(X_train.shape)
      #  print(X_test.shape)
        
      #  print("=============X_train========")
      #  print(X_train.style)
      #  print(X_train)
      #  print("=============X_test========")
      #  print(X_test)
        
        
        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        print(y_test_predicted)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        print("Mean squared error: %.2f"
                % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))
        print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
         #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId','place','lbs','finishTime']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        print(dataset_test_withPredicted)
       
        #break;


20190217
20190217 1 2000 4


NameError: name 'features' is not defined

In [ ]:
  predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )

In [47]:
 #explained_variance_score(y_test, y_test_predicted)
df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
dataset_test_toshow=dataset_test[['rateDate','raceSeqOfDate','distance','raceClass','horseId','place','winOdds','lbw','finishTime']]
dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
dataset_test_withPredicted

,rateDate,raceSeqOfDate,distance,raceClass,horseId,place,winOdds,lbw,finishTime,finishTime_predicted
13560,20190123,8,1650,3,B085,3,4.1,0.50,100.09,100.021965
13558,20190123,8,1650,3,A228,1,3.2,0.00,100.02,100.046780
13559,20190123,8,1650,3,A240,2,4.3,0.15,100.05,100.390609
13562,20190123,8,1650,3,B223,5,5.6,2.00,100.33,100.393883
13564,20190123,8,1650,3,A189,7,37.0,3.00,100.49,100.498137
13565,20190123,8,1650,3,A362,8,21.0,5.25,100.85,100.502601
13566,20190123,8,1650,3,V279,9,20.0,5.75,100.95,100.622227
13563,20190123,8,1650,3,A247,6,15.0,2.75,100.48,100.655952
13561,20190123,8,1650,3,V379,4,42.0,1.50,100.26,100.683624
13568,20190123,8,1650,3,T098,11,86.0,9.50,101.56,100.695285
